<a href="https://colab.research.google.com/github/Art9050/Test_Work/blob/main/%D0%A2%D0%B8%D0%BD%D1%8C%D0%BA_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Загрузка и подключение к базе данных

In [ ]:
!pip install SQLAlchemy==2.0
%load_ext sql

In [ ]:
%config SqlMagic.displaylimit = 10

In [ ]:
# Необходимо написать SQL запрос, который выведет все
# активные вклады с балансом более 10 тысяч евро или 11
# тысяч долларов. Не подразумевается суммирование балансов
# для одного клиента и конвертация валюты.
%%sql
select * from Dep
where status  = 'active'
and (1=0
or (balance > 10000 and currency = 'EUR')
or (balance > 11000 and currency = 'USD')
)

In [ ]:
# С именем Алина. Возраст по убыванию
%%sql
SELECT * FROM contact
WHERE client LIKE '%Алина%'
order by age DESC

In [ ]:
%%sql
SELECT avg(age) FROM contact;

--SELECT CAST(AVG(age*1.0) AS NUMERIC(6,2)) FROM contact;

In [ ]:
# Сумма всех счетов для лиц возрастом за 25
%%sql
select distinct client_nm from(
    SELECT
        Dep.client,
        balance,
        SUM(balance) OVER(PARTITION BY contact.client) AS 'Sum',
        contact.client as client_nm,
        age
    FROM contact
    left join Dep
    on contact.client_id = Dep.client
)
where age > 25

In [ ]:
# Необходимо написать SQL запрос, который выводит имя клиента и
# напротив имени клиента сумму всех вкладов клиента. Сумма должна
# быть в рублях. Если необходимо конвертировать валюту, то можно
# воспользоваться курсом на момент решения задачи.
%%sql
select
    client_nm,
    SUM(balance) OVER(PARTITION BY client_nm) AS 'Sum_Dep_RUR'
from(
    SELECT
        Dep.client,
        CASE
            WHEN currency = 'EUR' THEN balance/102,47
            WHEN currency = 'USD' THEN balance/96,54
            WHEN currency = 'GBP' THEN balance/120,78
            ELSE balance
        END as balance,
        contact.client as client_nm,
    FROM contact
    left join Dep
    on contact.client_id = Dep.client
)

In [ ]:
# Необходимо написать SQL запрос, который выводит идентификаторы
# клиентов, у которых нет счёта. А также идентификаторы тех
# клиентов, которые зафиксированы в таблице Dep, но отсутствуют
# в таблице Contact. В результате необходимо получить список идентификаторов.
%%sql
SELECT
    Dep.client as cl_id
FROM Dep
left join Dep
    on Dep.client = contact.client_id
Where contact.client is NULL
UNION aLL
SELECT
    Dep.client as cl_id
FROM Dep
right join Dep
    on Dep.client = contact.client_id
Where contact.client is NULL